In [1]:
import azureml.core
from azureml.core import Workspace, Datastore

ws = Workspace.from_config('./demoProject/config.json')
ws

Workspace.create(name='IntellipaatDemos', subscription_id='567af235-69fa-493d-abb8-e70b312cce1c', resource_group='Testgroup')

In [2]:

datastore = Datastore.get(ws, 'demostore')

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
compute_name = "bestcomputer"
vm_size = "Standard_E4s_v3"
compute_target = ws.compute_targets[compute_name]


In [4]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()
# `compute_target` as defined in "Azure Machine Learning compute" section above
aml_run_config.target = compute_target

USE_CURATED_ENV = True
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn','matplotlib'], 
        pip_packages=['azureml-sdk', 'azureml-dataset-runtime[fuse,pandas]'], 
        pin_sdk_version=False)


In [15]:
from azureml.pipeline.steps import PythonScriptStep
source_dir = ""
entry_point = "Titanic_Preds.py"

In [16]:
py_script_run_step = PythonScriptStep(
    source_directory = source_dir,
    script_name=entry_point,
    compute_target=compute_target,
    runconfig=aml_run_config,
    allow_reuse=False
)

In [17]:
pipeline_steps = [py_script_run_step ]

In [18]:
from azureml.pipeline.core import Pipeline

# Build the pipeline
pipeline1 = Pipeline(workspace=ws, steps=[pipeline_steps])

In [19]:
from azureml.core import Experiment

# Submit the pipeline to be run
pipeline_run1 = Experiment(ws, 'first_try_of_titanic_pipeline').submit(pipeline1)
pipeline_run1.wait_for_completion(show_output = True)


Created step Titanic_Preds.py [5dabcaaa][3e5ab9bd-57dd-4502-a4e5-fe408c51de02], (This step will run and generate new outputs)
Submitted PipelineRun 2a906bd0-6b19-4229-b328-e24d305a7d03
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2a906bd0-6b19-4229-b328-e24d305a7d03?wsid=/subscriptions/567af235-69fa-493d-abb8-e70b312cce1c/resourcegroups/Testgroup/workspaces/IntellipaatDemos&tid=5afbb968-8544-491f-90e1-9161db08035c
PipelineRunId: 2a906bd0-6b19-4229-b328-e24d305a7d03
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2a906bd0-6b19-4229-b328-e24d305a7d03?wsid=/subscriptions/567af235-69fa-493d-abb8-e70b312cce1c/resourcegroups/Testgroup/workspaces/IntellipaatDemos&tid=5afbb968-8544-491f-90e1-9161db08035c
PipelineRun Status: Running


StepRunId: 3554ed51-8320-407e-bad3-ae0ac502ebbd
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/3554ed51-8320-407e-bad3-ae0ac502ebbd?wsid=/subscriptions/567af235-69fa-493d-abb8-e70b312cce1c/resourcegroups/Testg